<a id="top"></a>
# Custom-calibrated STIS spectra

***

## Learning Goals

By the end of this tutorial, you will:

- learn how the ULLYSES team creates custom-calibrated STIS spectra
- learn how to create generic custom-calibrated STIS spectra 

## Table of Contents
**0. [Introduction](#introduction)**

**1. [Obtain Data](#sec1)**

**2. [Inspect default MAST 1D spectra](#sec2)**

\- 2.1 [STIS fringing](#sec21)

\- 2.2 [Data Quality flags](#sec22) 

**3. [STIS YAML files](#sec3)**

**4. [Reprocessing data](#sec4)**

**5. [Creating your own custom STIS spectra](#sec5)**

<a id="introduction"></a>
## Introduction

The Hubble Space Telescope’s (HST) Ultraviolet Legacy Library of Young Stars as Essential Standards ([ULLYSES](https://ullyses.stsci.edu/index.html)) program has devoted approximately 1,000 HST orbits to the production of an ultraviolet spectroscopic library of young high- and low-mass stars in the local universe. The ULLYSES team produces several types of High Level Science Products (HLSPs). Products are made using both HST, FUSE, and LCO data.

For HST/STIS long-slit spectra, custom calibration procedures are applied to improve upon the default 1D spectra. This can include custom DQ-flagging, extraction (when needed), and fringe correction for NIR data.

In this notebook we will show how custom STIS spectra are created, and how to create them yourself.

### Imports
We need only access to basic python packages, `astropy` for reading FITS files, `matplotlib` for plotting, and ULLYSES packages (`ullyses` and `ullyses_utils`) to create custom-calibrated STIS spectra.

In [ ]:
import sys
current_stdout = sys.stdout
import os
import glob
import pprint
from collections import Counter

from astropy.io import fits

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=13,6
#plt.style.use('seaborn-v0_8-notebook')
plt.style.use('tableau-colorblind10')

from ullyses.calibrate_stis_data import calibrate_stis_data
from ullyses import make_stis_x1ds
import ullyses_utils
from ullyses_utils.readwrite_yaml import read_config

***

<a id="sec1"></a>
## Obtain Data

To download data, we will retrieve it directly from MAST. For this example, we'll download the STIS observations of ULLYSES T Tauri Star SZ-76. This includes one science exposure for each of the following gratings: STIS/G230L, STIS/G430L, and STIS/G750L. For the NUV-MAMA observation (G230L), we download the `flt` file (flat-fielded 2D image), while for the CCD observations (G430L & G750L), we download the `flt` and `crj` files (flat-fielded and cosmic ray-rejectd 2D image). For both MAMA and CCD observations, we also retrieve the default 1D spectral files for comparison (for CCD, these are `sx1` files, for MAMA, `x1d`). Finally, in addition, we download the `raw` contemporaneous fringeflat associated with the G750L exposure.

These datasets will download into a new directory in your current working directory, called `notebook_download/`.

In [ ]:
!curl -L -X GET "https://mast.stsci.edu/search/hst/api/v0.1/retrieve_product?product_name=OEIMDS010%2Foeimds010_flt.fits" --output "notebook_download/oeimds010_flt.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/search/hst/api/v0.1/retrieve_product?product_name=OEIMDS010%2Foeimds010_spt.fits" --output "notebook_download/oeimds010_spt.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/search/hst/api/v0.1/retrieve_product?product_name=OEIMDS020%2Foeimds020_crj.fits" --output "notebook_download/oeimds020_crj.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/search/hst/api/v0.1/retrieve_product?product_name=OEIMDS020%2Foeimds020_flt.fits" --output "notebook_download/oeimds020_flt.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/search/hst/api/v0.1/retrieve_product?product_name=OEIMDS020%2Foeimds020_spt.fits" --output "notebook_download/oeimds020_spt.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/search/hst/api/v0.1/retrieve_product?product_name=OEIMDS030%2Foeimds030_crj.fits" --output "notebook_download/oeimds030_crj.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/search/hst/api/v0.1/retrieve_product?product_name=OEIMDS030%2Foeimds030_flt.fits" --output "notebook_download/oeimds030_flt.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/search/hst/api/v0.1/retrieve_product?product_name=OEIMDS030%2Foeimds030_spt.fits" --output "notebook_download/oeimds030_spt.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/search/hst/api/v0.1/retrieve_product?product_name=OEIMDS040%2Foeimds040_raw.fits" --output "notebook_download/oeimds040_raw.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/search/hst/api/v0.1/retrieve_product?product_name=OEIMDS040%2Foeimds040_spt.fits" --output "notebook_download/oeimds040_spt.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/search/hst/api/v0.1/retrieve_product?product_name=OEIMDS010%2Foeimds010_x1d.fits" --output "notebook_download/oeimds010_x1d.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/search/hst/api/v0.1/retrieve_product?product_name=OEIMDS020%2Foeimds020_sx1.fits" --output "notebook_download/oeimds020_sx1.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/search/hst/api/v0.1/retrieve_product?product_name=OEIMDS030%2Foeimds030_sx1.fits" --output "notebook_download/oeimds030_sx1.fits" --fail --create-dirs

***

<a id="sec2"></a>
## Inspect default MAST 1D spectra

Let's take a look at the default MAST 1D spectra for these datasets.

In [ ]:
# Glob the x1ds and sx1s
orig_x1ds = glob.glob("notebook_download/*x1d.fits") + glob.glob("notebook_download/*sx1.fits")
orig_x1ds.sort()

In [ ]:
for item in orig_x1ds:
    # Get ext=1 SCI data
    data = fits.getdata(item)
    grating = fits.getval(item, "opt_elem")
    plt.plot(data["wavelength"].flatten(), data["flux"].flatten(), alpha=0.7, label=f"{grating} {os.path.basename(item)}")
    plt.legend()
    plt.ylim(-2e-15, 2.0e-14)
    plt.title("Original 1D spectra")
    plt.xlabel("Wavelength [A]")
    plt.ylabel("Flux [ergs/s/cm^2/A]")

At first glance nothing the spectra above don't look too bad. But let's look at two aspects in particular: DQ (data quality) flags, and the NIR region. First, let's zoom in on the NIR region of the G750L spectrum (`oeimds030_sx1.fits`).

<a id="sec21"></a>
### STIS fringing

In [ ]:
# Open just the G750L 1D spectrum and plot it
g750l = fits.getdata("notebook_download/oeimds030_sx1.fits")
plt.plot(g750l["wavelength"].flatten(), g750l["flux"].flatten())
plt.title("G750L oeil3s030_sx1.fits")
plt.xlabel("Wavelength [A]")
plt.ylabel("Flux [ergs/s/cm^2/A]")
plt.xlim(left=8000)
plt.ylim(3e-15, 2e-14)

Something different is happening here: [fringing](https://www.stsci.edu/files/live/sites/www/files/home/hst/instrumentation/stis/documentation/instrument-science-reports/_documents/199819.pdf). This is caused by intereference of multiple reflections between the two surfaces of the CCD. Fortunately, there is a method to correct such an effect-- By obtaining a contemporaneous fringe flat using the same grating, the fringe pattern can be modeled and divided from the science spectrum. 

<a id="sec22"></a>
### Data Quality flags

Now let's look at the Data Quality (DQ) flags. [DQ flags](https://hst-docs.stsci.edu/stisdhb/chapter-2-stis-data-structure/2-5-error-and-data-quality-array#id-2.5ErrorandDataQualityArray-2.5.2DataQualityFlagging) denote when something potentially problematic occurred at a specific pixel location or time. Flags can mark effects such as bad pixels, cosmic rays, or saturation. DQ flags are set as specific bits in a 16-bit integer word. Some DQ flags are informational, or do not greatly affect science data. Conversely, some flags are so serious that affected data should not be used-- these values are included in the `SDQFLAGS` value (serious data quality flags). The `SDQFLAGS` value can be obtained from the 1st STIS header. Performing a bitwise AND on a given DQ flag with `SDQFLAGS` will reveal if the flag of interest is considered serious.

Let's look at the serious DQ flags for each STIS spectrum.

In [ ]:
for item in orig_x1ds:
    data = fits.getdata(item)
    grating = fits.getval(item, "opt_elem")
    # Get the DQ array from the BinTable
    dq = data["dq"]
    # Get SDQFLAGS from the 1st header
    sdqflags = fits.getval(item, "sdqflags", 1)
    # Perform a bitwise AND to find indices which have serious DQ flags
    bad_pix = np.where(dq & sdqflags != 0)

    plt.plot(data["wavelength"][0], data["flux"][0], alpha=0.7, label=f"{grating} {os.path.basename(item)}")
    # Now mark the indices with serious DQ flags as X's
    plt.plot(data["wavelength"][bad_pix], data["flux"][bad_pix], "rx", alpha=.3)
    plt.ylim(-2e-15, 2.0e-14)
    plt.legend()
    plt.title("Original 1D spectra")
    plt.xlabel("Wavelength [A]")
    plt.ylabel("Flux [ergs/s/cm^2/A]")

Everything marked with a red X is considered to have a serious data quality flag. You may notice that the CCD spectra (G430L & G750L) are greatly affected by this, while the MAMA spectrum is not (G230L). The ULLYSES coaddition algorithm discards all spectral elements with DQ flags that are in SDQFLAGS, so a non-negligible portion of the CCD spectra would be discarded in this case. Affected spectral elements are zeroed out, which would give us spectra that look like the figure below. 

In [ ]:
for item in orig_x1ds:
    data = fits.getdata(item)
    grating = fits.getval(item, "opt_elem")
    dq = data["dq"]
    sdqflags = fits.getval(item, "sdqflags", 1)
    bad_pix = np.where(dq & sdqflags != 0)
    # Zero out the indices with serious DQ flags
    data["flux"][bad_pix] = 0
    
    plt.plot(data["wavelength"][0], data["flux"][0], alpha=0.7, label=f"{grating} {os.path.basename(item)}")
    plt.ylim(-2e-15, 2.0e-14)
    plt.legend()
    plt.title("Original 1D spectra")
    plt.xlabel("Wavelength [A]")
    plt.ylabel("Flux [ergs/s/cm^2/A]")

This seriously degrades the quality of the data. Let's look a little closer at which flag in particular is causing problems...

In [ ]:
for item in orig_x1ds:
    data = fits.getdata(item)
    c = Counter(data["dq"][0])
    print(f"DQ flags for {os.path.basename(item)}:")
    pprint.pprint(c)
    print("")

You can see that the vast majority of serious data quality flags are coming from the DQ=16 value. (A DQ value of 0 means there are not data quality issues.) Referring to the [STIS Instrument Handbook](https://hst-docs.stsci.edu/stisdhb/chapter-2-stis-data-structure/2-5-error-and-data-quality-array#id-2.5ErrorandDataQualityArray-2.5.2DataQualityFlagging), DQ=16 indicates that the pixel has a high dark rate. To mitigate this, the ULLYSES team manually determines pixels with a high dark rate by using a modified threshold compared to the CalSTIS pipeline.

Both effects-- fringing and overflagging-- are corrected by the ULLYSES calibration wrapper, as shown in the following sections.

<a id="sec3"></a>
## STIS yaml files

To perform the custom calibration needed for ULLYSES STIS data, we need to specify what corrections are required. To do this, the necessary modifications are recorded in a YAML file. YAML files are convenient, human-readable files that act as configuration setups-- in this case, they describe the non-standard calibration parameters needed to rectify STIS data. 

All YAML files used by the ULLYSES team are available in the [`ullyses_utils` package](https://github.com/spacetelescope/ullyses-utils/tree/main/src/ullyses_utils/data/stis_configs). Each STIS exposure was manually inspected by the ULLYSES team to account for any irregularities in the data. 

Let's take a look at the YAML files for SZ-76 now.

In [ ]:
# Find your local installation of the ullyses_utils package
utils_dir = ullyses_utils.__path__[0]
# Get the path to the STIS YAML files
yamls = glob.glob(os.path.join(utils_dir, f"data/stis_configs/*sz-76*.yaml"))
yamls.sort()
pprint.pprint(yamls)

There is one YAML file for each observed grating. Let's take a look at each one.

In [ ]:
for yaml in yamls:
    # Use the ULLYSES convenience function to read each YAML file
    config = read_config(yaml)
    print(f"{os.path.basename(yaml)}:")
    pprint.pprint(config)
    print("")

Each YAML file includes information about special calibration parameters needed to obtain optimized 1D spectra. Note the extra defringing parameters for the G750L data.

<a id="sec4"></a>
## Reprocessing data

To rectify the data, we must reprocess it using the custom ULLYSES wrapper around CalSTIS. This wrapper can perform any number of corrections as specified in an input YAML file. This includes: 
- Removing the fringe pattern. Using the [`stistools` defringing](https://stistools.readthedocs.io/en/latest/defringe_guide.html) package, the amplitude, shift, and position of the fringes are determined for each science spectrum. This is done by processing the contemporaneous fringe flat taken right before or after each G750L or G750M science exposure.
- Manually calculating the number of pixels with high dark rate (DQ=16), using a modified threshold.
- Performing a custom extraction to one or more targets in the long slit. This is done e.g. when the target is especially faint and requires a smaller extraction box, when the target is miscentered in the slit, or there is a companion source in the slit.

Let's reprocess the SZ-76 data by using the YAML files as input to the ULLYSES code.

In [ ]:
# Define the output directory
outdir = "custom_products"

# The target name must be lowercase!
make_stis_x1ds.make_custom_x1ds(datadir="notebook_download", outdir=outdir, targ="sz-76")

# This and the method above yield identical results
#for yaml in yamls:
#    calibrate_stis_data(indir="notebook_download", yamlfile=yaml, outdir=outdir)

In [ ]:
# We have to reset the stdout, since the CalSTIS log modifies it
sys.stdout = current_stdout

<a id="sec41"></a>
### Inspect output products

Running the custom ULLYSES STIS calibration above not only produced new 1D spectra, but also diagnostic figures and log files. You can walk through what was done by examining the log printed above. A summary of the applied non-standard calibrations is printed at the end of the run for each input YAML file/science exposure.

Let's take a look at the output products now.

In [ ]:
ls custom_products/

Let's look at how the new 1D spectra compare to the default MAST ones. Comparison figures are already created by the code we just ran, so let's open those. You can also open them outside of this notebook if you prefer.

In [ ]:
from IPython.display import display_pdf
with open('custom_products/sz-76_g230l_oeimds010_1d_compare.pdf', "rb") as f:
    display_pdf(f.read(),raw=True)

In [ ]:
from IPython.display import display_pdf
with open('custom_products/sz-76_g430l_oeimds020_1d_compare.pdf', "rb") as f:
    display_pdf(f.read(),raw=True)

In [ ]:
from IPython.display import display_pdf
with open('custom_products/sz-76_g750l_oeimds030_1d_compare.pdf', "rb") as f:
    display_pdf(f.read(),raw=True)

A few things to note: the improvement for G230L is minimal to nonexistent, but there is a marked difference for the CCD datasets. For both G430L and G750L, then number of spectral elements marked as having a serious DQ flag has dramatically decreased. This means that the ULLYSES coaddition algorithm will no longer have the large number of dropouts we saw before. For G750L, the NIR fringing has been successfully removed.

Let's plot up all the new spectra to see the full picture.

In [ ]:
# One figure for new spectra, one for original
fig1,ax1 = plt.subplots()
fig2,ax2 = plt.subplots()

for item in orig_x1ds:
    orig_data = fits.getdata(item)
    new_x1d = glob.glob(os.path.join(f"custom_products/{os.path.basename(item)[:9]}*x1d.fits"))[0]
    new_data = fits.getdata(new_x1d)
    grating = fits.getval(item, "opt_elem")
    sdqflags = fits.getval(item, "sdqflags", 1)
    orig_bad_pix = np.where(orig_data["dq"] & sdqflags != 0)
    new_bad_pix = np.where(new_data["dq"] & sdqflags != 0)
    
    ax1.plot(orig_data["wavelength"][0], orig_data["flux"][0], alpha=0.7, label=f"{grating} {os.path.basename(item)}")
    ax1.plot(orig_data["wavelength"][orig_bad_pix], orig_data["flux"][orig_bad_pix], "rx", alpha=0.3)
    ax2.plot(new_data["wavelength"][0], new_data["flux"][0], alpha=0.7, label=f"{grating} {os.path.basename(new_x1d)}")
    ax2.plot(new_data["wavelength"][new_bad_pix], new_data["flux"][new_bad_pix], "rx", alpha=0.3)
ax1.legend()
ax1.set_title("Original Spectra")
ax1.set_ylim(-2e-15, 2.0e-14)
ax2.legend()
ax2.set_title("Custom Spectra")
ax2.set_ylim(-2e-15, 2.0e-14)

<a id="sec5"></a>
## Creating your own custom STIS spectra

To create your own custom STIS spectra, you must first create new input YAML files as required by the ULLYSES code. You can find a [template form](https://github.com/spacetelescope/ullyses-utils/blob/main/src/ullyses_utils/data/stis_configs/target_grating.yaml) in the same `ullyses_utils` repo, as shown below. You would simply need to modify specific parameters as required. The name of the YAML file must follow the format `<target>_<grating>.yaml`, and be lowercase, in order to be properly processed *automatically*.

The template is shown below, along with comments that explain various parameters.

In [ ]:
utils_dir = ullyses_utils.__path__[0]
template = os.path.join(utils_dir, f"data/stis_configs/target_grating.yaml")
with open(template, "r") as f:
    contents = f.read()
    print(contents)

***

## Additional Resources

- [ULLYSES](https://ullyses.stsci.edu)
- [MAST API](https://mast.stsci.edu/api/v0/index.html)

## About this Notebook
For support, contact us at the [ULLYSES Helpdesk](https://stsci.service-now.com/hst?id=sc_cat_item&sys_id=a3b8ec5edbb7985033b55dd5ce961990&sysparm_category=ac85189bdb4683c033b55dd5ce96199c).

**Author:**  Jo Taylor \
**Updated On:** March 7 2024

## Citations
* See the [ULLYSES website](https://ullyses.stsci.edu/ullyses-cite.html) for citation guidelines.


***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 